In [35]:
import pandas as pd
import requests
import bs4 as bs
import lxml
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os
import zipfile
import io


In [36]:
# I have to download this file and merge the headings to make the urls to scrape from
# this is the zip file https://www1.nyc.gov/assets/finance/downloads/tar/avroll_19.zip  It is a microsoft access file,
# I downlaoded it and turned it into a csv
taxroll1 = pd.read_csv("avroll1234.csv")

drop all un needed columns/ only keep borough address number street address & bbl# data dictionary available here : https://www1.nyc.gov/assets/finance/downloads/tar/tarfieldcodes.pdf

Key

AV-BORO Length 1 numeric BORO CODES. 1 = MANHATTAN 2 = BRONX 3 = BROOKLYN 4 = QUEENS 5 = STATEN ISLAND

AV-HOUSENUM-LO Length 12 Character contains the lowest house number of the property, right justified padded with leading zeros, except when blank

AV-HOUSENUM-HI Length 12 Character contains the highest house number of the property may be identical to the house number low

AV-STREET-NAME Length 20 Character Street name for the property

Other useful

AV-BLOCK Length 5 numeric VALID BLOCK RANGES BY BORO MANHATTAN 1 TO 2,255 BRONX 2,260 TO 5,958 BROOKLYN 1 TO 8,955 QUEENS 1 TO 16,350 STATEN ISLAND 1 TO 8,050

AV-OWNER Length 21 Character The Owner’s Name.

AV-TOTAL-UNITS Length 5 numeric (no decimals) Total number of units in the building

AV-RESIDENTIAL-UNITS Length 5 numeric (no decimals) Number of residential units in building

In [37]:
taxroll1.head()

,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1000010010,1,1,10,NaN,GOVERNORS ISLAND CORP,Y4,4,177,0,...,0,0,104445450.0,157532850.0,104445450.0,157532850.0,NaN,FINAL,2018/19,AC-TR
1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,1046,...,0,0,4225950.0,12148920.0,4225950.0,12148920.0,NaN,FINAL,2018/19,AC-TR
2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,63,0,...,0,0,14972400.0,107838180.0,14972400.0,107838180.0,NaN,FINAL,2018/19,AC-TR
3,1000010301,1,1,301,NaN,NaN,U0,4,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,FINAL,2018/19,AC-TR
4,1000010401,1,1,401,NaN,NaN,U0,4,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,FINAL,2018/19,AC-TR


In [38]:
taxroll1.columns

Index(['BBLE', 'B', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL', 'TAXCLASS',
       'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND', 'AVTOT',
       'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL', 'BLDFRONT',
       'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2', 'EXCD2', 'PERIOD',
       'YEAR', 'VALTYPE'],
      dtype='object')

In [39]:
taxroll1["B"] = taxroll1["B"].astype(str)

In [40]:
taxroll1["B"].replace(['1', '2', '3', '4'], ['Manhattan', 'Bronx', 'Brooklyn', 'Queens'], inplace=True)

In [41]:
#split addresses into two columns
taxroll1['Address-Number'], taxroll1['Address-Name'] = taxroll1['STADDR'].str.split(' ', 1).str

In [44]:
#replace all spaces in address name with %20
taxroll1['Address-Name']  = taxroll1['Address-Name'].str.replace(' ', '%20')

In [54]:
taxroll1.head()

,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,LTDEPTH,...,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE,Address-Number,Address-Name,URL
0,1000010010,Manhattan,1,10,NaN,GOVERNORS ISLAND CORP,Y4,4,177,0,...,157532850.0,104445450.0,157532850.0,NaN,FINAL,2018/19,AC-TR,1,GOVERNORS%20ISLAND,http://whoownswhat.justfix.nyc/address/Manhatt...
1,1000010101,Manhattan,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,1046,...,12148920.0,4225950.0,12148920.0,NaN,FINAL,2018/19,AC-TR,1,LIBERTY%20ISLAND,http://whoownswhat.justfix.nyc/address/Manhatt...
2,1000010201,Manhattan,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,63,0,...,107838180.0,14972400.0,107838180.0,NaN,FINAL,2018/19,AC-TR,1,ELLIS%20ISLAND,http://whoownswhat.justfix.nyc/address/Manhatt...
3,1000010301,Manhattan,1,301,NaN,NaN,U0,4,0,0,...,NaN,NaN,NaN,NaN,FINAL,2018/19,AC-TR,JOE,DIMAGGIO%20HIGHWAY,http://whoownswhat.justfix.nyc/address/Manhatt...
4,1000010401,Manhattan,1,401,NaN,NaN,U0,4,0,0,...,NaN,NaN,NaN,NaN,FINAL,2018/19,AC-TR,JOE,DIMAGGIO%20HIGHWAY,http://whoownswhat.justfix.nyc/address/Manhatt...


adapt the columns so that there is one that contains a 


http://whoownswhat.justfix.nyc/address/ ["B'] / ['Address-Number'] / ['Address-Name']


In [53]:
#create new column that contains the URL
taxroll1["URL"] = "http://whoownswhat.justfix.nyc/address/" + taxroll1['B'].map(str) + taxroll1['Address-Number'] + "/" + taxroll1['Address-Name']

In [55]:
#run code in Selenium, Match the data scrape with each column